In [1]:
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama

from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

from langchain_core.runnables import RunnablePassthrough

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate





USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Set up the documents to work with - read in and split

pdf_file = "./IPCC_AR6_WGII_Chapter02.pdf"
pdf_file = "./IPCC_AR6_SYR_SPM.pdf"
loader = PyPDFLoader(pdf_file)
data = loader.load()
print(f"data {data[:200]}")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

ValueError: File path ./IPCC_AR6_SYR_SPM.pdf is not a valid file or url

In [ ]:
# Create local vector store based on the the split documents
local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)
retriever = vectorstore.as_retriever()

# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [ ]:
# Demonstrate that the local vector store can do something useful - finding fragments that are similar to a question
question = "how has anthropogenic climate change affected wildfires"
docs = vectorstore.similarity_search(question)
for d in docs:
    print(f">> {d}\n")

In [ ]:
model = ChatOllama(
    model="llama3:latest",
    # model="llama3.1:8b", ## This was default from tutorial
)

In [ ]:
# Interaction template

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

In [ ]:

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

In [ ]:
print(qa_chain.invoke("How has human action affected wildfires?"))

Human action has increased wildfires by intensifying heat, drying out vegetation, and accelerating burning due to climate change. Additionally, human land use has driven changes in wildfire trends globally, with some regions experiencing more significant impacts than others. Overall, human influence on fire ignition is evident through documented decreases on holy days and traditional religious days of rest.

In [ ]:
print("xxx")
print("yyy")